In [6]:
# ============================================
# ATIVIDADE 2.3 — Interpretação com SHAP
# ============================================

import os
import pandas as pd
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
import shap
import matplotlib.pyplot as plt
import sys

# ============================================
# ETAPA 1 — Preparação dos Dados
# ============================================

# Usar caminho local dentro do projeto
base_path = "dataset"

# Criar pasta se não existir
os.makedirs(base_path, exist_ok=True)

# Caminho do arquivo CSV com os dados PCA + coluna alvo
csv_path = os.path.join(base_path, "dataset_com_pca.csv")

# Carregar dataset
try:
    df = pd.read_csv(csv_path)
except FileNotFoundError:
    print(f"❌ Arquivo não encontrado em: {csv_path}")
    sys.exit(1)

# Mostrar colunas disponíveis
print("📋 Colunas encontradas no dataset:")
print(df.columns.tolist())

# Detectar a variável alvo
target_col = None
for col in df.columns:
    if col.strip().lower() in ["class", "target", "fraude", "label", "y"]:
        target_col = col
        break

if not target_col:
    print("❌ Nenhuma coluna reconhecida como variável alvo ('Class', 'Target', etc.) foi encontrada.")
    sys.exit(1)

# Separar features (X) e target (y)
X = df.drop(columns=[target_col])
y = df[target_col]

# Dividir em treino e teste
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, stratify=y, random_state=42
)

# ============================================
# ETAPA 2 — Treinar o Modelo XGBoost
# ============================================

modelo_xgb = XGBClassifier(eval_metric='logloss', random_state=42)
modelo_xgb.fit(X_train, y_train)

# ============================================
# ETAPA 3 — Explicabilidade com SHAP
# ============================================

explainer = shap.Explainer(modelo_xgb, X_train)
shap_values = explainer(X_test)

# ============================================
# ETAPA 4 — Geração de Gráficos
# ============================================

# 1. Importância global
shap.summary_plot(shap_values, X_test, show=False)
plt.savefig(f"{base_path}/shap_summary_plot.png", bbox_inches='tight')
plt.close()

# 2. Gráfico de dependência da 'V14', se existir
if "V14" in X_test.columns:
    shap.dependence_plot("V14", shap_values.values, X_test, show=False)
    plt.savefig(f"{base_path}/shap_dependence_V14.png", bbox_inches='tight')
    plt.close()

# 3. Waterfall plot para uma predição
shap.initjs()
plt.figure()
shap.plots.waterfall(shap_values[0])
plt.savefig(f"{base_path}/shap_force_plot_exemplo.png", bbox_inches='tight')
plt.close()

# ============================================
# Conclusão
# ============================================

print("\n✅ SHAP executado com sucesso!")
print(f"🔍 Coluna alvo detectada: '{target_col}'")
print(f"📁 Gráficos salvos na pasta: {base_path}")


❌ Arquivo não encontrado em: dataset\dataset_com_pca.csv


SystemExit: 1